<a href="https://colab.research.google.com/github/alessiomodonesi/Python-Exercises/blob/main/ai/lab6/Intelligenza_Artificiale_Lab6_BayesianNetwork_new_versione_pomegranate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Laboratory Lecture 6: Additional materials - Implementation with new version of pomegranate**

---

## EXAMPLE 1: Excel for Filtering (Day 1)

The spreadsheet <a href="https://docs.google.com/spreadsheets/d/1BeARmsvIf-pHuuPgICcgyTTOqZq8yC9-/edit?usp=share_link&ouid=116452643742209360003&rtpof=true&sd=true">Umbrella.xlsx</a> models the umbrella example over the first 2 days.


<img src='https://drive.google.com/uc?id=1QkA4DpBqRRl5bc9opUvqm16S30_a-Npd'>


On the top line, the probability of rain for Day 0 is the prior probability.

At the bottom of the sheet are the conditional probability tables for the transition model and the sensor model.

The predicted probability for rain on Day 1 (top) is computed from the probability for Day 0 and the transition model.

To get the filtered probability, we have to bring in information about whether we saw an umbrella or not. The filtered probability of rain for Day 1 (middle of the sheet) is computed by combining the predicted probability for
Day 1, the sensor model, and what we know about umbrellas.

<b>Note:</b> There are two versions of the filtered probability. The raw values which we get directly from the
calculation, and the normalized values (raw values sclaed so they add to 1).

Look at what happens if you change the probability of <tt>umbrella/not umbrella</tt>. Currently the values say
you see an umbrella (probability of <tt>umbrella</tt> is 1 and that of <tt>not umbrella</tt> is 0).
What happens if you don't see an umbrella (probability of <tt>umbrella</tt> is 0 and that of <tt>not umbrella</tt> is 1)?

What about if you have no information (probability of <tt>umbrella</tt> is 0.5 and that of <tt>not umbrella</tt> is 0.5)?


The column for Day 2 just repeats the calculations for Day 1, but starting from the results from Day 1.

Thus the predicted probability for Day 2 is calculated by applying the transition model to the (normalized) filtered probability for Day 1.

The filtered probability of Day 2 is calculated from the predicted probability for Day 2, the sensor model, and what we know about umbrellas.

In other words, the probabilities for Day 2 are computed just like those for Day 1. The calculation is modular.

Look at what happens when the probabilities of umbrella/not umbrella on Days 1 and 2 vary

Pomegranate for Predicting and Filtering

For this example we will use a Python package called <tt>pomegranate</tt>, which provides support for probabilistic reasoning. If you have not installed it before, you will need to do so with:

<tt>pip install pomegranate</tt>

Then you can run the following version of the umbrella model. <tt>pomegranate</tt> can only solve Bayesian neworks (not Dynamic Bayesian Networks), so we have to unroll the whole example to the depth that we want.

In [ ]:
!pip install numpy
!pip install pomegranate #NEW VERSION


In [ ]:
import warnings
warnings.filterwarnings("ignore") # Suppresses future warnings about MaskedTensor creation

from typing import Iterable

import torch
from pomegranate.bayesian_network import BayesianNetwork
from pomegranate.distributions import Categorical, ConditionalCategorical
from torch import nan
from torch.masked import MaskedTensor


# Variables are RainN and UmbrellaN for N = 0, 1, 2, ...
# We have a prior for Rain0, two values: 0=no and 1=yes

# Prior distribution for Rain0: P(Rain0) = [0.5, 0.5]
Rain0 = Categorical([[0.5, 0.5]])

# Transition model: P(Rain_t+1 | Rain_t)
# Format: [[P(n|n), P(y|n)],
#          [P(n|y), P(y|y)]]
# Rain persists with prob 0.7, changes with prob 0.3
Rain1 = ConditionalCategorical([[
    [0.7, 0.3],  # P(Rain1=n|Rain0=n)=0.7, P(Rain1=y|Rain0=n)=0.3
    [0.3, 0.7]   # P(Rain1=n|Rain0=y)=0.3, P(Rain1=y|Rain0=y)=0.7
]])

Rain2 = ConditionalCategorical([[
    [0.7, 0.3],
    [0.3, 0.7]
]])

# Sensor model: P(Umbrella | Rain)
# Format: [[P(n|n), P(y|n)],
#          [P(n|y), P(y|y)]]
# If rain: umbrella seen with prob 0.9
# If no rain: umbrella seen with prob 0.2
Umbrella1 = ConditionalCategorical([[
    [0.8, 0.2],  # P(Umbrella1=n|Rain1=n)=0.8, P(Umbrella1=y|Rain1=n)=0.2
    [0.1, 0.9]   # P(Umbrella1=n|Rain1=y)=0.1, P(Umbrella1=y|Rain1=y)=0.9
]])

Umbrella2 = ConditionalCategorical([[
    [0.8, 0.2],
    [0.1, 0.9]
]])

# Create the Bayesian Network
model = BayesianNetwork()

# Add all distributions
model.add_distributions([Rain0, Rain1, Umbrella1, Rain2, Umbrella2])

# Add edges to define the structure
model.add_edge(Rain0, Rain1)
model.add_edge(Rain1, Umbrella1)
model.add_edge(Rain1, Rain2)
model.add_edge(Rain2, Umbrella2)

print("Model created successfully!")
print(f"Number of nodes: {len(model.distributions)}")

Model created successfully!
Number of nodes: 5


Now that we have the model entered, we can ask it questions. We can first ask it to predict the probability of rain on days 1 and 2:

In [ ]:
def predict_probabilitites(model: BayesianNetwork, scenario: Iterable[Iterable]):
    scenario = torch.tensor(scenario)
    scenario = MaskedTensor(scenario, mask=~torch.isnan(scenario)).long()
    results = model.predict_proba(scenario)
    return [r.numpy() for r in results]

In [ ]:
# Do not instantiate any of the variables:
scenario = [[nan, nan, nan, nan, nan]] # nan = not observed variable
# Run the model
results =  predict_probabilitites(model,scenario)

# Ask for the probability of rain on Day 1:
print(f"P(~Rain1)={results[1][0][0]}   P(Rain1)={results[1][0][1]}")
# And the probability of rain on Day 2:
print(f"P(~Rain2)={results[3][0][0]}   P(Rain2)={results[3][0][1]}")

P(~Rain1)=0.5   P(Rain1)=0.5
P(~Rain2)=0.5   P(Rain2)=0.5


So both Day 1 and Day 2 have a probability 0.5 of being rainy before we see any umbrellas.

In Bayesian probability terms, this tells us that we can't say anything about how likely it is to rain
(a binary variable with probability of 0.5 for both values is how we represent ¯\_(ツ)_/¯ ).



The reason that we ask for elements 1 and 3 of the datastructure <tt>results</tt> is because they are
elements 1 and 3 of
<tt>model.add_states(s1, s2, s3, s4, s5)</tt>

Now let's tell the model that we see an umbrella on Day 1 and see what that gets us:

In [ ]:
# Set Umbrella on Rain1 to 'y'
scenario = [[nan, nan, 1, nan, nan]]
# Run the model
results =  predict_probabilitites(model,scenario)

# Ask for the probability of rain on Day 1:
print(f"P(~Rain1|Umbrella1)={results[1][0][0]:.2f}   P(Rain1|Umbrella1)={results[1][0][1]:.2f}")
# And the probability of rain on Day 2:
print(f"P(~Rain2|Umbrella1)={results[3][0][0]:.2f}   P(Rain2|Umbrella1)={results[3][0][1]:.2f}")

P(~Rain1|Umbrella1)=0.18   P(Rain1|Umbrella1)=0.82
P(~Rain2|Umbrella1)=0.37   P(Rain2|Umbrella1)=0.63


So it has filtered the probability of rain for Day 1, and also predicted the probability for Day 2 as well.

That is because pomegranate propagates all updates through the whole model/network. It has, for example also computed the probability of rain on Day 0 (that it rained on Day 0 even though we said noting about the rain that day):

In [ ]:
# Ask for the probability of rain on Day 0:
print(f"P(~Rain0|Umbrella1)={results[0][0][0]:.2f}   P(Rain0|Umbrella1)={results[0][0][1]:.2f}")

P(~Rain0|Umbrella1)=0.37   P(Rain0|Umbrella1)=0.63


This is what we call the smoothed probability of rain on Day 0.

Now let's tell <tt>pomegranate</tt> about rain on Day 3, so we need to add information about Day3:

In [ ]:
Rain3 = ConditionalCategorical([[
    [0.7, 0.3],
    [0.3, 0.7]
]])

nodes = list(model.distributions) + [Rain3]
edges = list(model.edges) + [(Rain2, Rain3)]
model2 = BayesianNetwork(nodes, edges) # you can also instantiate a model by directly providing nodes and edges

# Umbrellas on Day 1 and 2:
scenario = [[nan, nan, 1, nan, 1,nan]]
# Run the model
results =  predict_probabilitites(model2,scenario)

# Ask for the probability of rain on Day 1:
print(f"P(~Rain1|Umbrella1,Umbrella2)={results[1][0][0]:.2f}   P(Rain1|Umbrella1,Umbrella2)={results[1][0][1]:.2f}")
# And the probability of rain on Day 2:
print(f"P(~Rain2|Umbrella1,Umbrella2)={results[3][0][0]:.2f}   P(Rain2|Umbrella1,Umbrella2)={results[3][0][1]:.2f}")

P(~Rain1|Umbrella1,Umbrella2)=0.12   P(Rain1|Umbrella1,Umbrella2)=0.88
P(~Rain2|Umbrella1,Umbrella2)=0.12   P(Rain2|Umbrella1,Umbrella2)=0.88


Note that we didn't tell pomegranate to do smoothing. As we saw before with Day 0, it (in effect) always runs the backwards propagation and gives us smoothed probabilities for all days before the latest piece of evidence.

I said "in effect" because pomegranate doesn't do the computation the way we studied. It just computes the probability of every hidden variable given the evidence.